<a href="https://colab.research.google.com/github/bipinKrishnan/torchkeras/blob/master/Model_layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pkbar
#!pip install barbar

In [344]:
import torch
from torch import nn
from torch import optim
import pkbar
from tqdm.notebook import tqdm

# TorchKeras

In [345]:
class Input():
  def __init__(self, shape):
    self.size = shape

In [347]:
class Dense_(nn.Module):
  def __init__(self, outputs, activation):
    super().__init__()
    self.outputs = outputs
    self.activation = activation

  def __call__(self, input_size):
    self.inputs_size = input_size.size
    self.layers = nn.Sequential(
        nn.Linear(self.inputs_size, self.outputs),
        self.activation
    )

    return self.layers

class Dense(nn.Module):
  def __init__(self, outputs, activation):
    super().__init__()
    self.outputs = outputs
    self.activation = activation

  def __call__(self, inputs):
    self.inputs = inputs
    self.layers = list(inputs)
    self.layers.extend([nn.Linear(self.layers[-2].out_features, self.outputs), self.activation])

    self.layers = nn.Sequential(*self.layers)

    return self.layers


In [348]:
class Model(Input):
  def __init__(self, inputs, outputs):
    self.input_size = inputs.size
    self.model = outputs

  def compile(self, optimizer, loss, metrics):
    self.opt = optimizer
    self.criterion = loss
    self.metrics = metrics

  def summary(self):
    return self.model

  def fit(self, data_x, data_y, epochs):
    self.model.train()

    for epoch in range(epochs):
      print("Epoch {}/{}".format(epoch+1, epochs))
      progress = pkbar.Kbar(target=len(data_x), width=25)
      
      for i, (data, target) in enumerate(zip(data_x, data_y)):
        self.opt.zero_grad()

        train_out = self.model(data)
        loss = self.criterion(train_out, target)
        loss.backward()

        self.opt.step()

        progress.update(i, values=[("loss: ", loss.item())])

      progress.add(1)

  def evaluate(self, test_x, test_y):
    self.model.eval()
    correct, loss = 0.0, 0.0

    progress = pkbar.Kbar(target=len(test_x), width=25)

    for i, (data, target) in enumerate(zip(test_x, test_y)):
      out = self.model(data)
      loss += self.criterion(out, target)

      correct += (torch.max(out, 1)[1] == target).sum()

      progress.update(i, values=[("loss", loss.item()), ("acc", (correct/len(test_x)).item())])
    progress.add(1)


  def fit_generator(self, generator, epochs):
    self.model.train()

    for epoch in range(epochs):
      print("Epoch {}/{}".format(epoch+1, epochs))
      progress = pkbar.Kbar(target=len(generator), width=25)

      for i, (data, target) in enumerate(generator):
        self.opt.zero_grad()

        train_out = self.model(data)
        loss = self.criterion(train_out.squeeze(), target)
        loss.backward()

        self.opt.step()

        progress.update(i, values=[("loss: ", loss.item())])

      progress.add(1)
      

  def evaluate_generator(self, generator):
    self.model.eval()
    correct, loss = 0.0, 0.0

    progress = pkbar.Kbar(target=len(generator), width=25)

    for i, (data, target) in enumerate(generator):
      out = self.model(data)
      loss += self.criterion(out.squeeze(), target)

      correct += (torch.max(out.squeeze(), 1)[1] == target).sum()

      progress.update(i, values=[("test_acc", (correct/len(generator)).item()), ("test_loss", loss.item())])

    progress.add(1)
      

# Test data

In [358]:
input = Input(shape=1024)
x = Dense_(54, nn.ReLU())(input)
y = Dense(34, nn.ReLU())(x)
z = Dense(43, activation=nn.ReLU())(y)
a = Dense(10, activation=nn.ReLU())(z)
b = Dense(10, activation=nn.ReLU())(a)

In [359]:
model = Model(input, b)
model.compile(optim.Adam(z.parameters(), lr=0.001), nn.CrossEntropyLoss(), 'accuracy')

In [360]:
model.summary()

Sequential(
  (0): Linear(in_features=1024, out_features=54, bias=True)
  (1): ReLU()
  (2): Linear(in_features=54, out_features=34, bias=True)
  (3): ReLU()
  (4): Linear(in_features=34, out_features=43, bias=True)
  (5): ReLU()
  (6): Linear(in_features=43, out_features=10, bias=True)
  (7): ReLU()
  (8): Linear(in_features=10, out_features=10, bias=True)
  (9): ReLU()
)

In [361]:
torch.manual_seed(42)

x = torch.rand((10, 1, 1024), dtype=torch.float)
#y = torch.tensor((torch.rand(128, 1) < 0.5), dtype=torch.float)
y = torch.tensor([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]).reshape(-1, 1)

In [362]:
model.fit(x, y, 4)

Epoch 1/4
10/10 [=========================] - 0s 5ms/step - loss: : 2.3293
Epoch 2/4
10/10 [=========================] - 0s 2ms/step - loss: : 2.3118
Epoch 3/4
10/10 [=========================] - 0s 2ms/step - loss: : 2.3065
Epoch 4/4
10/10 [=========================] - 0s 2ms/step - loss: : 2.2974


In [363]:
model.evaluate(x, y)

10/10 [=========================] - 0s 1ms/step - loss: 13.6983 - acc: 0.1111   


# CIFAR100

In [364]:
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

import numpy as np
import matplotlib.pyplot as plt

In [375]:
bs = 128

transform = transforms.Compose([transforms.Grayscale(num_output_channels=1),
                                transforms.Resize((1, 28*28)),
                                transforms.ToTensor()
])

trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


In [376]:
#images for feeding fit_generator dense network 
#must be of the form (1, 1, height*width*channel)
trainset[0][0].shape

torch.Size([1, 1, 784])

In [377]:
trainloader = DataLoader(trainset, batch_size=bs, shuffle=True)
testloader = DataLoader(testset, batch_size=bs)

In [379]:
input = Input(shape=784)
x = Dense_(54, nn.ReLU())(input)
y = Dense(34, nn.ReLU())(x)
z = Dense(43, activation=nn.ReLU())(y)
a = Dense(10, activation=nn.ReLU())(z)
b = Dense(100, activation=nn.ReLU())(a)

model = Model(input, b)
model.compile(optim.Adam(b.parameters(), lr=0.001), nn.CrossEntropyLoss(), 'accuracy')
model.summary()

Sequential(
  (0): Linear(in_features=784, out_features=54, bias=True)
  (1): ReLU()
  (2): Linear(in_features=54, out_features=34, bias=True)
  (3): ReLU()
  (4): Linear(in_features=34, out_features=43, bias=True)
  (5): ReLU()
  (6): Linear(in_features=43, out_features=10, bias=True)
  (7): ReLU()
  (8): Linear(in_features=10, out_features=100, bias=True)
  (9): ReLU()
)

In [380]:
model.fit_generator(trainloader, 5)

Epoch 1/5
391/391 [=========================] - 15s 39ms/step - loss: : 4.6044
Epoch 2/5
391/391 [=========================] - 16s 40ms/step - loss: : 4.5768
Epoch 3/5
391/391 [=========================] - 16s 40ms/step - loss: : 4.5635
Epoch 4/5
391/391 [=========================] - 16s 40ms/step - loss: : 4.5518
Epoch 5/5
391/391 [=========================] - 16s 40ms/step - loss: : 4.5394


In [381]:
model.evaluate_generator(testloader)

79/79 [=========================] - 3s 37ms/step - test_acc: 2.1699 - test_loss: 183.8001


# CIFAR10

In [384]:
bs = 128

transform = transforms.Compose([transforms.Grayscale(num_output_channels=1),
                                transforms.Resize((1, 28*28)),
                                transforms.ToTensor()
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [385]:
#images for feeding fit_generator dense network 
#must be of the form (1, 1, height*width*channel)
trainset[0][0].shape

torch.Size([1, 1, 784])

In [386]:
trainloader = DataLoader(trainset, batch_size=bs, shuffle=True)
testloader = DataLoader(testset, batch_size=bs)

In [387]:
input = Input(shape=784)
x = Dense_(54, nn.ReLU())(input)
y = Dense(34, nn.ReLU())(x)
z = Dense(43, activation=nn.ReLU())(y)
a = Dense(10, activation=nn.ReLU())(z)
b = Dense(100, activation=nn.ReLU())(a)

model = Model(input, b)
model.compile(optim.Adam(b.parameters(), lr=0.001), nn.CrossEntropyLoss(), 'accuracy')
model.summary()

Sequential(
  (0): Linear(in_features=784, out_features=54, bias=True)
  (1): ReLU()
  (2): Linear(in_features=54, out_features=34, bias=True)
  (3): ReLU()
  (4): Linear(in_features=34, out_features=43, bias=True)
  (5): ReLU()
  (6): Linear(in_features=43, out_features=10, bias=True)
  (7): ReLU()
  (8): Linear(in_features=10, out_features=100, bias=True)
  (9): ReLU()
)

In [388]:
model.fit_generator(trainloader, 5)

Epoch 1/5
391/391 [=========================] - 15s 39ms/step - loss: : 3.8513
Epoch 2/5
391/391 [=========================] - 16s 40ms/step - loss: : 3.6157
Epoch 3/5
391/391 [=========================] - 15s 39ms/step - loss: : 3.5921
Epoch 4/5
391/391 [=========================] - 18s 46ms/step - loss: : 3.5657
Epoch 5/5
391/391 [=========================] - 16s 40ms/step - loss: : 3.5509


In [389]:
model.evaluate_generator(testloader)

79/79 [=========================] - 3s 39ms/step - test_acc: 8.4667 - test_loss: 143.4968


# MNIST

In [395]:
bs = 32

transform = transforms.Compose([
                                transforms.Resize((1, 28*28)),
                                transforms.ToTensor()
])

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

In [396]:
#images for feeding fit_generator dense network 
#must be of the form (1, 1, height*width*channel)
trainset[0][0].shape

torch.Size([1, 1, 784])

In [397]:
trainloader = DataLoader(trainset, batch_size=bs, shuffle=True)
testloader = DataLoader(testset, batch_size=bs)

In [398]:
input = Input(shape=784)
x = Dense_(54, nn.ReLU())(input)
y = Dense(34, nn.ReLU())(x)
z = Dense(43, activation=nn.ReLU())(y)
a = Dense(10, activation=nn.ReLU())(z)
b = Dense(100, activation=nn.ReLU())(a)

model = Model(input, b)
model.compile(optim.Adam(b.parameters(), lr=0.001), nn.CrossEntropyLoss(), 'accuracy')
model.summary()

Sequential(
  (0): Linear(in_features=784, out_features=54, bias=True)
  (1): ReLU()
  (2): Linear(in_features=54, out_features=34, bias=True)
  (3): ReLU()
  (4): Linear(in_features=34, out_features=43, bias=True)
  (5): ReLU()
  (6): Linear(in_features=43, out_features=10, bias=True)
  (7): ReLU()
  (8): Linear(in_features=10, out_features=100, bias=True)
  (9): ReLU()
)

In [399]:
model.fit_generator(trainloader, 5)

Epoch 1/5
1875/1875 [=========================] - 20s 10ms/step - loss: : 2.4547
Epoch 2/5
1875/1875 [=========================] - 20s 11ms/step - loss: : 1.9142
Epoch 3/5
1875/1875 [=========================] - 20s 11ms/step - loss: : 1.5841
Epoch 4/5
1875/1875 [=========================] - 20s 10ms/step - loss: : 1.2916
Epoch 5/5
1875/1875 [=========================] - 20s 11ms/step - loss: : 1.2442


In [400]:
model.evaluate_generator(testloader)

313/313 [=========================] - 3s 8ms/step - test_acc: 9.5785 - test_loss: 185.5959
